In [1]:
import requests
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import boto3
from datetime import datetime
import os
from io import BytesIO

In [2]:
link2 = f"https://api.openweathermap.org/data/2.5/weather?q=novo hamburgo,br&APPID=550d57361bde0d30b0dd389950c781ad"

In [3]:
link2

'https://api.openweathermap.org/data/2.5/weather?q=novo hamburgo,br&APPID=550d57361bde0d30b0dd389950c781ad'

In [4]:
requisicao = requests.get(link2)

In [5]:
requisicao.json()

{'coord': {'lon': -51.1306, 'lat': -29.6783},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04n'}],
 'base': 'stations',
 'main': {'temp': 294.91,
  'feels_like': 295.54,
  'temp_min': 293.95,
  'temp_max': 296,
  'pressure': 1011,
  'humidity': 92,
  'sea_level': 1011,
  'grnd_level': 1006},
 'visibility': 10000,
 'wind': {'speed': 1.01, 'deg': 40, 'gust': 1.31},
 'clouds': {'all': 100},
 'dt': 1700009131,
 'sys': {'type': 2,
  'id': 2020383,
  'country': 'BR',
  'sunrise': 1699950106,
  'sunset': 1699998975},
 'timezone': -10800,
 'id': 3456068,
 'name': 'Novo Hamburgo',
 'cod': 200}

In [6]:
df = requisicao.json()

In [7]:
df1 = pd.json_normalize(df)

In [8]:
df_lon_lat = pd.DataFrame(df['coord'], index=[0])

In [9]:
df_lon_lat

,lon,lat
0,-51.1306,-29.6783


In [10]:
df_weather = pd.DataFrame(df['weather'], index=[0])

In [11]:
df_weather

,id,main,description,icon
0,804,Clouds,overcast clouds,04n


In [12]:
df_main = pd.DataFrame(df['main'], index=[0])

In [13]:
df_main

,temp,feels_like,temp_min,temp_max,pressure,humidity,sea_level,grnd_level
0,294.91,295.54,293.95,296,1011,92,1011,1006


In [14]:
df1 = pd.DataFrame(df1)

In [15]:
df1

,weather,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,...,main.grnd_level,wind.speed,wind.deg,wind.gust,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,"[{'id': 804, 'main': 'Clouds', 'description': ...",stations,10000,1700009131,-10800,3456068,Novo Hamburgo,200,-51.1306,-29.6783,...,1006,1.01,40,1.31,100,2,2020383,BR,1699950106,1699998975


In [16]:
selected_columns = [
    "base",
    "visibility",
    "dt",
    "timezone",
    "id",
    "name",
    "cod",
    "coord.lon",
    "coord.lat",
    "main.temp",
    "main.feels_like",
    "main.temp_min",
    "main.temp_max",
    "main.pressure",
    "main.humidity",
    "main.sea_level",
    "main.grnd_level",
    "wind.speed",
    "wind.deg",
    "wind.gust",
    "clouds.all",
    "sys.type",
    "sys.id",
    "sys.country",
    "sys.sunrise",
    "sys.sunset",
]

In [17]:
df_clumns_selected = df1[selected_columns]

In [18]:
df_clumns_selected

,base,visibility,dt,timezone,id,name,cod,coord.lon,coord.lat,main.temp,...,main.grnd_level,wind.speed,wind.deg,wind.gust,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,stations,10000,1700009131,-10800,3456068,Novo Hamburgo,200,-51.1306,-29.6783,294.91,...,1006,1.01,40,1.31,100,2,2020383,BR,1699950106,1699998975


In [19]:
super_df = pd.concat([df_weather, df_lon_lat, df_main, df_clumns_selected], axis=0, ignore_index=True)

In [20]:
super_df.to_parquet("C:/Users/crist/Desktop/s3_testes/super_df.parquet")

In [21]:
super_df

,id,main,description,icon,lon,lat,temp,feels_like,temp_min,temp_max,...,main.grnd_level,wind.speed,wind.deg,wind.gust,clouds.all,sys.type,sys.id,sys.country,sys.sunrise,sys.sunset
0,804.0,Clouds,overcast clouds,04n,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,-51.1306,-29.6783,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,294.91,295.54,293.95,296.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3456068.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1006.0,1.01,40.0,1.31,100.0,2.0,2020383.0,BR,1.699950e+09,1.699999e+09


putaria com o boto3

In [22]:
# Defina suas credenciais explicitamente (não recomendado)
profile_name = 'CRISTIAN_AWS'

s3 = boto3.session.Session(profile_name=profile_name).client('s3')

In [23]:
bucket_name = 'cristian-teste-buckt-from-python-code'

In [24]:
local_file_path = "C:/Users/crist/Desktop/s3_testes/super_df.parquet"

In [25]:
dia_da_exucucao = datetime.now().strftime('%Y-%m-%d-%H-%M-%S')

In [26]:
dia_da_exucucao

'2023-11-14-21-45-33'

In [27]:
s3_file_name = f'df_teste_{dia_da_exucucao}.parquet'

In [28]:
s3_file_name

'df_teste_2023-11-14-21-45-33.parquet'

In [29]:
s3.upload_file(local_file_path, bucket_name, s3_file_name)

---------------------------------

CATUABA